In [1]:
#%%
import torch
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
from torch import nn

# 读取数据
train_dir='/home/tsinghuaee112/Classification/DataFewShot/Train'
val_dir='/home/tsinghuaee112/Classification/DataFewShot/Val'
transform  = transforms.Compose([
         transforms.Resize((64,64)),
         #transforms.RandomHorizontalFlip(),
         transforms.ToTensor(),
         transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.4, 0.4, 0.4]),
])#读取数据时对数据进行初步处理：大小归一化，值强度归一化
trainset = ImageFolder(train_dir, transform=transform)
valset=ImageFolder(val_dir,transform=transform)



In [2]:
#%%
#数据划分batch
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=2)
valloader = torch.utils.data.DataLoader(valset, batch_size=32,
                                         shuffle=True, num_workers=2)
classes = ('p1','p3','p6','p9','p12','p14','p17','p19','p22','p25','p27')


In [3]:

#%%

#网络模型

#class Net(nn.Module):
#    def __init__(self):
#        super(Net, self).__init__()
#        self.conv1 = nn.Conv2d(3, 6, 5)
#        self.pool = nn.MaxPool2d(2, 2)
#        self.conv2 = nn.Conv2d(6, 16, 5)
#        self.fc1 = nn.Linear(16 * 5 * 5, 120)
#        self.fc2 = nn.Linear(120, 84)
#        self.fc3 = nn.Linear(84, 19) #输出层

#    def forward(self, x):
#        x = self.pool(F.relu(self.conv1(x)))
#        x = self.pool(F.relu(self.conv2(x)))
#        x = x.view(-1, 16 * 5 * 5)
#        x = F.relu(self.fc1(x))
#        x = F.relu(self.fc2(x))
#        x = self.fc3(x)
#        return x

#net = Net()



PATH = './Resnet_net.pth'
net=models.resnet18(pretrained=True)
#net = models.resnext50_32x4d(pretrained=True)
num_ftrs=net.fc.in_features
net.fc=nn.Linear(num_ftrs,19)
net.load_state_dict(torch.load(PATH,map_location=lambda storage,loc: storage))
net.fc=nn.Linear(num_ftrs,11)

In [4]:

import torch.optim as optim
#定义优化模型
criterion = nn.CrossEntropyLoss()

#optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
#scheduler=optim.lr_scheduler.StepLR(optimizer,step_size=10000,gamma=0.2)


ignored_params= list(map(id,net.fc.parameters()))
base_params=filter(lambda p: id(p) not in ignored_params, net.parameters())
optimizer=torch.optim.SGD([
    {'params':base_params},
    {'params':net.fc.parameters(),'lr':0.001}
],lr=0,momentum=0.9)


#%%

import os
#os.environ["CUDA_VISIBLE_DEVICES"]="2"
#net=net.cuda()
#训练网络
for epoch in range(40):  # loop over the dataset multiple times
    running_loss = 0.0
#    for i, data in enumerate(trainloader, 0):

    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        #inputs=inputs.cuda()
        #labels=labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()

        optimizer.step()
        #scheduler.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')

Finished Training


In [5]:
#%%
PATH = './Resnet_fewshotnet.pth'
torch.save(net.state_dict(), PATH)
#dataiter = iter(valloader)
#images, labels = dataiter.next()

#第1个batch的分类结果
#print (images)
#print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))
#net = Net()
net=models.resnet18(pretrained=True)
#net = models.resnext50_32x4d(pretrained=True)
num_ftrs=net.fc.in_features
net.fc=nn.Linear(num_ftrs,11)
net.load_state_dict(torch.load(PATH))
#outputs = net(images)
#_, predicted = torch.max(outputs, 1)
#print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
#                              for j in range(32)))


<All keys matched successfully>

In [6]:

#统计正确率
correct = 0
total = 0
with torch.no_grad():
    for data in valloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

#%%
#统计各类别的正确率
class_correct = list(0. for i in range(11))
class_total = list(0. for i in range(11))
with torch.no_grad():
    for data in valloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        for i in range(labels.shape[0]):
            label = labels[i]
            class_correct[label] += (predicted[i] == labels[i]).sum().item()
            class_total[label] += 1

for i in range(11):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of the network on the test images: 59 %
Accuracy of    p1 : 25 %
Accuracy of    p3 : 80 %
Accuracy of    p6 : 70 %
Accuracy of    p9 : 63 %
Accuracy of   p12 : 60 %
Accuracy of   p14 : 70 %
Accuracy of   p17 : 50 %
Accuracy of   p19 : 95 %
Accuracy of   p22 : 50 %
Accuracy of   p25 : 47 %
Accuracy of   p27 : 20 %
